In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import xgboost as xgb
#https://habrahabr.ru/company/ods/blog/327242/

import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
%matplotlib inline
import seaborn as sb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import MinMaxScaler
from pandas.tseries.offsets import *

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import __version__
from plotly import graph_objs as go

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

C:\Users\TEMP\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#Читаем данные
df=pd.read_pickle('MG_Sales.pickle',compression='gzip')
celebrate=pd.read_pickle('celebrate.pickle')

In [3]:
#готовим выборку обучения
begin_period=dt.datetime(2015,1,1)
end_period=dt.datetime(2017,4,30)


days=30
start_forecast=end_period+dt.timedelta(days=1)
end_forecast=start_forecast+dt.timedelta(days=days)

#query="Дата>=20150101 and ТоварЦеноваяГруппа=='9000-12000' and ЦветМеталла=='Красное золото' and Коллекция=='Гранаты с фианитом' and Магазин=='Гранд Ривер Астрахань'"
query="Дата>=20150101 and Магазин=='РИО Дмитровка'"
#('9000-12000', 'Красное золото', 'Гранаты с фианитом', 'Гранд Ривер Астрахань')
time_series=pd.DataFrame(data=df.query(query).groupby('Дата')['Количество'].sum())
time_series.index.name='ds'
time_series.columns=['y']
time_series.y=time_series.y

#Пустой период в днях обучающей выборки
delta=end_forecast-begin_period+dt.timedelta(days=1)
dummy_train = pd.DataFrame(index=np.array([begin_period + dt.timedelta(days=x) for x in range(0, delta.days)]).astype('datetime64[D]'))
dummy_train.index.name='ds'

#сливаем обучающую выборку и пустой период чтобы избежать пропусков дат, пропуски заполняем нулями
time_series=dummy_train.merge(time_series,left_index=True, right_index=True,how='outer').fillna(0)  

In [50]:
class modelForecast:
    def __init__(self, celebrate):
        self.cel=celebrate
        
        
    def setPercentil(self,row,percentiles):
        if row<percentiles['llim']: return -2
        if row>percentiles['ulim']: return 2
        if row<percentiles['med'][0]: return -1
        if row>percentiles['med'][1]: return 1
        return 0

    def getPercentil(self,ts):
        #Фиксируем аномально низкие и высокие продажи
        ul=5#Персентиль высоких продаж 2
        ll=5#7#Персентиль низких продаж 10
        md=25#ширина медианы
        percentiles={}

        percentiles['ulim']=np.percentile(ts['y'], 100.-ul)
        percentiles['llim']=np.percentile(ts['y'], ll)
        percentiles['med']=np.percentile(ts['y'], [50-md,50+md])
        return percentiles


    #временные характеристики
    def setNewValues(self,_ts):
        ts=_ts.copy()
        y=None
        
        ts['День недели'] = ts.index.weekday
        ts['Выходной'] = ((ts.index.weekday.isin([5,6]))|(ts.index.isin(self.cel['Праздник'])))*1
        ts['Неделя'] = ts.index.week
        ts['Год'] = ts.index.year
        ts['Месяц'] = ts.index.month
        ts['День месяца'] = ts.index.day


        if 'y' in ts.columns:#ТОЛЬКО для обучающей выборки
            #Фомируем характеристики модели 
            percentiles=self.getPercentil(ts)#получаем распределение персентилей


            ts['Среднее по дню года']=0
            ts['Максимальное по дню года']=0
            ts['Минимальное по дню года']=0
            #Среднее по дню года, нарастающим
            for year in ts['Год'].unique():    
                _mean_day_year=ts.loc[ts['Год']<=year,['Месяц','День месяца','y']].reset_index()#.drop('ds',axis=1)

                mean2829=np.mean(_mean_day_year.loc[(_mean_day_year['Месяц']==2)&(_mean_day_year['День месяца'].isin([28,29])),'y'])
                mean2801=np.mean(_mean_day_year.loc[(_mean_day_year['Месяц']==2)&(_mean_day_year['День месяца']==28)|(_mean_day_year['Месяц']==3)&(_mean_day_year['День месяца']==1),'y'])

                _mean_day_year.loc[(_mean_day_year['Месяц']==2)&(_mean_day_year['День месяца']==28),'y']=mean2829
                _mean_day_year.loc[(_mean_day_year['Месяц']==2)&(_mean_day_year['День месяца']==29),'y']=mean2801   

                mean_day_year=dict(_mean_day_year.groupby(['Месяц','День месяца'])['y'].mean())
                ts['Среднее по дню года']=ts.apply(lambda row: mean_day_year[row['Месяц'],row['День месяца']] if row['Год']==year else row['Среднее по дню года'] , axis=1)             
                max_day_year=dict(_mean_day_year.groupby(['Месяц','День месяца'])['y'].max())
                ts['Максимальное по дню года']=ts.apply(lambda row: max_day_year[row['Месяц'],row['День месяца']] if row['Год']==year else row['Максимальное по дню года'] , axis=1)
                min_day_year=dict(_mean_day_year.groupby(['Месяц','День месяца'])['y'].min())
                ts['Минимальное по дню года']=ts.apply(lambda row: min_day_year[row['Месяц'],row['День месяца']] if row['Год']==year else row['Минимальное по дню года'] , axis=1)
                
            ts['Квантили']=ts.apply(lambda row: self.setPercentil(row['Среднее по дню года'],percentiles), axis=1)


            #порядок дней в сезонности недельной продажи за исключением аномалий
            week_d=pd.DataFrame(data=ts[ts['Квантили']==0].groupby('День недели')['y'].sum().sort_values())
            week_d.insert(0,'Недельная сезонность',list(range(week_d.shape[0])))
            for i in list(set(range(7))-set(week_d.index.values)):
                week_d.loc[i,'Недельная сезонность']=-1
            ts=self.weekseason(ts,week_d)

            #Среднее и тренды                        
            mean_dict=dict(ts.groupby(['Год'])['y'].mean())
            ts['Среднее за год']=ts.apply(lambda row: mean_dict[row['Год']] , axis=1)        

            #РАССЧИТЫВАЕМ коэффициент роста
            mean_month=ts.groupby(['Год','Месяц'])['y'].mean()
            mean_month_shift=mean_month.shift(12).fillna(0)
            inc_temp=(mean_month/mean_month_shift-1)#*mean_month.shift(11).fillna(0)        
            last_tempo=inc_temp.tail(1).values[0]

            #Вычленяем целевую переменную
            y=ts.y
            ts.drop(['y'], axis=1, inplace=True)

            #заглушки        
            ts.drop(['День недели'], axis=1, inplace=True)                

            self.week_d=week_d
            self.mean_day_year=mean_day_year
            self.percentiles=percentiles
            self.last_tempo=last_tempo            
            self.last_vals=ts.tail(1)
            self.columns=ts.columns
            self.max_day_year=max_day_year
            self.min_day_year=min_day_year
        return ts,y


    def weekseason(self,ts,week_d):
        ts['Недельная сезонность']=ts['День недели'].map(lambda cell: week_d.loc[cell,'Недельная сезонность'])
        return ts

    #Сдвигаем период на год вперед
    def fillTimeSeriesForecast(self,_tsf):
        tsf=self.weekseason(_tsf,self.week_d)
        tsf.drop(['День недели'], axis=1, inplace=True)

        tsf['Среднее по дню года']=tsf.apply(lambda row: self.mean_day_year[row['Месяц'],row['День месяца']], axis=1)
        tsf['Квантили']=tsf.apply(lambda row: self.setPercentil(row['Среднее по дню года'],self.percentiles), axis=1)

        tsf['Максимальное по дню года']=tsf.apply(lambda row: self.max_day_year[row['Месяц'],row['День месяца']], axis=1)
        tsf['Минимальное по дню года']=tsf.apply(lambda row: self.min_day_year[row['Месяц'],row['День месяца']], axis=1)
        
        
        #Мы знаем средние данные за год, учитываем как тренд        
        for col in ['Среднее за год']:
            tsf[col]=self.last_vals[col].values[0]
        return tsf[self.columns]#Колонки в правильном порядке


    def createTimeSeriesForecast(self,begin, end):
        date_list = pd.date_range(begin, end).tolist()
        time_series_forecast=pd.DataFrame(index=date_list)
        time_series_forecast.index.name='ds'    

        time_series_forecast,_=self.setNewValues(time_series_forecast)

        return self.fillTimeSeriesForecast(time_series_forecast)        

In [51]:
lastDay=end_period

new_model=modelForecast(celebrate)
time_series_train, y_train=new_model.setNewValues(time_series.loc[:lastDay])
    
tss = TimeSeriesSplit(n_splits=10)
tss_cv=list(tss.split(time_series_train,y_train))

# задаём параметры
params = {
        'objective': 'reg:linear',
        'booster':'gblinear',                
        'eta': 0.23,#коэффициент обучения
        'alpha': 0.1,        
        'eval_metric': 'rmse'
    }

#Нормализуйте обучающую выборку с помощью класса StandardScaler
scaler = StandardScaler(with_mean=True,with_std=True)
dtrain = xgb.DMatrix(scaler.fit_transform(time_series_train), label=y_train)#

# прогоняем на кросс-валидации с метрикой rmse
cv = xgb.cv(params, dtrain, metrics = ('rmse'), early_stopping_rounds=50,verbose_eval=False, show_stdv=False, num_boost_round=1500,folds=tss_cv)#,nfold=10

mod_n=cv['test-rmse-mean'].argmin()

deviation = cv.loc[mod_n]["test-rmse-mean"]
best_iteration=mod_n

#Модель предсказания
time_series_forecast=new_model.createTimeSeriesForecast(start_forecast, end_forecast)

In [52]:
bst = xgb.train(params, dtrain, best_iteration,verbose_eval=False)
prediction_test = pd.DataFrame(data=bst.predict(xgb.DMatrix(scaler.transform(time_series_forecast))),index=time_series_forecast.index)#
prediction_test[0]=prediction_test[0].map(lambda val: 0 if val<0.01 else round(val,0))

print('Средняя ошибка на кросс-валидации: ',deviation,', номер модели: ',mod_n)
print('Темп прироста по сравнению с прошлым годом: ',new_model.last_tempo)

y_test=time_series.loc[time_series_forecast.index]
print('RSME = ',np.sqrt(mean_squared_error(y_test.y,prediction_test[0])))

Средняя ошибка на кросс-валидации:  3.8723616 , номер модели:  38
Темп прироста по сравнению с прошлым годом:  -0.183486238532
RSME =  5.42098669747


In [53]:
y_test=time_series[start_forecast:end_forecast]


init_notebook_mode(connected = True)
trace1 = go.Scatter(
            x = prediction_test.index,
            y = prediction_test[0],
            mode = 'lines',            
            name = 'Прогноз 2017',
            line=dict(
                shape='spline',
                dash = 'dash',
                width = 3
            )
        )       

ty=(time_series[dt.datetime(2016,1,1):dt.datetime(2016,6,1)].shift(1,DateOffset(years=1)))
trace0 = go.Scatter(
            x = ty.index,
            y = ty.y,
            mode = 'lines',            
            name = '2016 год',
            line=dict(
                shape='spline',
                dash = 'dot'
            )
        )       

ty=(time_series[dt.datetime(2015,1,1):dt.datetime(2015,6,1)].shift(2,DateOffset(years=1)))
trace5 = go.Scatter(
            x = ty.index,
            y = ty.y,
            mode = 'lines',            
            name = '2015 год',
            line=dict(
                shape='spline',
                dash = 'dot'
            )
        )   


trace2 = go.Scatter(
            x = y_test.index,
            y = y_test.y,
            mode = 'lines',            
            name = 'Продажи 2017',
            line=dict(
                shape='spline'
            )
        ) 
trace4 = go.Scatter(
            x = y_test.index,
            y = y_test.y-prediction_test[0],
            mode = 'markers',
            name = 'Разница прогноза и данных',
            line=dict(
                shape='spline',
                dash = 'dot'
            )
        ) 

layout = go.Layout(
    title=query
)
    
fig =go.Figure(data = [trace1,trace2,trace5,trace0,trace4],layout=layout)#,trace0
iplot(fig, show_link=False)



In [ ]:
time_series.tail(31)

In [ ]:
time_series_forecast

In [ ]:
time_series.loc[time_series_forecast.index].tail(10)

In [ ]:
time_series.tail(10)

In [18]:
time_series_forecast

(            День недели  Выходной  Неделя   Год  Месяц  День месяца
 ds                                                                 
 2017-05-01            0         1      18  2017      5            1
 2017-05-02            1         0      18  2017      5            2
 2017-05-03            2         0      18  2017      5            3
 2017-05-04            3         0      18  2017      5            4
 2017-05-05            4         0      18  2017      5            5
 2017-05-06            5         1      18  2017      5            6
 2017-05-07            6         1      18  2017      5            7
 2017-05-08            0         1      19  2017      5            8
 2017-05-09            1         1      19  2017      5            9
 2017-05-10            2         0      19  2017      5           10
 2017-05-11            3         0      19  2017      5           11
 2017-05-12            4         0      19  2017      5           12
 2017-05-13            5         1

In [ ]:
ts=time_series.copy()
ts['Год'] = ts.index.year
ts['Месяц'] = ts.index.month

mean_month=ts.groupby(['Год','Месяц'])['y'].mean()
mean_month_shift=mean_month.shift(12).fillna(0)
inc_temp=(mean_month/mean_month_shift-1)#*mean_month.shift(11).fillna(0)        
inc_temp